# Titanic Baseline: LightGBM

First baseline experiment using LightGBM with basic feature engineering.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import lightgbm as lgb
import os

# Create experiments directory
os.makedirs('/home/code/experiments', exist_ok=True)

# Load data
train_df = pd.read_csv('/home/data/train.csv')
test_df = pd.read_csv('/home/data/test.csv')

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")
print("\nTrain columns:", train_df.columns.tolist())
print("\nTest columns:", test_df.columns.tolist())

Train shape: (891, 12)
Test shape: (418, 11)

Train columns: ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']

Test columns: ['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']


In [2]:
# Basic feature engineering
def engineer_features(df):
    df = df.copy()
    
    # Fill missing values
    df['Age'] = df['Age'].fillna(df['Age'].median())
    df['Fare'] = df['Fare'].fillna(df['Fare'].median())
    df['Embarked'] = df['Embarked'].fillna('S')
    
    # Extract titles from names
    df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    
    # Simplify titles
    title_mapping = {
        'Mr': 'Mr', 'Mrs': 'Mrs', 'Miss': 'Miss', 'Master': 'Master',
        'Dr': 'Other', 'Rev': 'Other', 'Col': 'Other', 'Major': 'Other',
        'Mlle': 'Miss', 'Countess': 'Other', 'Ms': 'Miss', 'Lady': 'Other',
        'Jonkheer': 'Other', 'Don': 'Other', 'Dona': 'Other', 'Mme': 'Mrs',
        'Capt': 'Other', 'Sir': 'Other'
    }
    df['Title'] = df['Title'].map(title_mapping)
    
    # Family features
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
    df['IsAlone'] = (df['FamilySize'] == 1).astype(int)
    
    # Age groups
    df['AgeGroup'] = pd.cut(df['Age'], bins=[0, 12, 18, 35, 60, 100], 
                           labels=['Child', 'Teen', 'YoungAdult', 'Adult', 'Senior'])
    
    # Fare per person
    df['FarePerPerson'] = df['Fare'] / df['FamilySize']
    
    # Cabin features
    df['HasCabin'] = df['Cabin'].notna().astype(int)
    df['CabinLetter'] = df['Cabin'].str[0]
    df['CabinLetter'] = df['CabinLetter'].fillna('Unknown')
    
    return df

# Apply feature engineering
train_processed = engineer_features(train_df)
test_processed = engineer_features(test_df)

print("Features engineered successfully")
print("\nSample of new features:")
print(train_processed[['Title', 'FamilySize', 'IsAlone', 'AgeGroup', 'FarePerPerson', 'HasCabin']].head())

Features engineered successfully

Sample of new features:
  Title  FamilySize  IsAlone    AgeGroup  FarePerPerson  HasCabin
0    Mr           2        0  YoungAdult        3.62500         0
1   Mrs           2        0       Adult       35.64165         1
2  Miss           1        1  YoungAdult        7.92500         0
3   Mrs           2        0  YoungAdult       26.55000         1
4    Mr           1        1  YoungAdult        8.05000         0


In [3]:
# Prepare data for modeling
# Define categorical and numerical features
categorical_features = ['Pclass', 'Sex', 'Embarked', 'Title', 'AgeGroup', 'CabinLetter']
numerical_features = ['Age', 'SibSp', 'Parch', 'Fare', 'FamilySize', 'FarePerPerson', 'HasCabin']

# Combine all features
feature_columns = categorical_features + numerical_features

# Create feature matrices
X = train_processed[feature_columns]
y = train_processed['Survived']
X_test = test_processed[feature_columns]

print(f"Training features shape: {X.shape}")
print(f"Test features shape: {X_test.shape}")
print(f"\nFeatures: {feature_columns}")

Training features shape: (891, 13)
Test features shape: (418, 13)

Features: ['Pclass', 'Sex', 'Embarked', 'Title', 'AgeGroup', 'CabinLetter', 'Age', 'SibSp', 'Parch', 'Fare', 'FamilySize', 'FarePerPerson', 'HasCabin']


In [4]:
# Encode categorical features
from sklearn.preprocessing import LabelEncoder

X_encoded = X.copy()
X_test_encoded = X_test.copy()

label_encoders = {}
for col in categorical_features:
    le = LabelEncoder()
    # Fit on combined data to handle unseen categories
    combined_data = pd.concat([X[col], X_test[col]], axis=0)
    le.fit(combined_data)
    
    X_encoded[col] = le.transform(X[col])
    X_test_encoded[col] = le.transform(X_test[col])
    
    label_encoders[col] = le

print("Categorical features encoded")
print("\nSample of encoded data:")
print(X_encoded.head())

Categorical features encoded

Sample of encoded data:
   Pclass  Sex  Embarked  Title  AgeGroup  CabinLetter   Age  SibSp  Parch  \
0       2    1         2      2         4            8  22.0      1      0   
1       0    0         0      3         0            2  38.0      1      0   
2       2    0         2      1         4            8  26.0      0      0   
3       0    0         2      3         4            2  35.0      1      0   
4       2    1         2      2         4            8  35.0      0      0   

      Fare  FamilySize  FarePerPerson  HasCabin  
0   7.2500           2        3.62500         0  
1  71.2833           2       35.64165         1  
2   7.9250           1        7.92500         0  
3  53.1000           2       26.55000         1  
4   8.0500           1        8.05000         0  


In [5]:
# Cross-validation setup
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Store results
cv_scores = []
oof_predictions = np.zeros(len(X))
test_predictions = np.zeros(len(X_test))

print(f"Starting {n_splits}-fold cross-validation...")

fold = 1
for train_idx, val_idx in skf.split(X_encoded, y):
    print(f"\nFold {fold}/{n_splits}")
    
    X_train, X_val = X_encoded.iloc[train_idx], X_encoded.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
    # Create LightGBM datasets
    train_data = lgb.Dataset(X_train, label=y_train)
    val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)
    
    # Parameters
    params = {
        'objective': 'binary',
        'metric': 'binary_error',
        'boosting_type': 'gbdt',
        'num_leaves': 31,
        'learning_rate': 0.05,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': -1,
        'seed': 42
    }
    
    # Train model
    model = lgb.train(
        params,
        train_data,
        num_boost_round=1000,
        valid_sets=[val_data],
        valid_names=['valid'],
        callbacks=[lgb.early_stopping(50), lgb.log_evaluation(0)]
    )
    
    # Predictions
    val_pred = model.predict(X_val, num_iteration=model.best_iteration)
    val_pred_binary = (val_pred > 0.5).astype(int)
    
    test_pred = model.predict(X_test_encoded, num_iteration=model.best_iteration)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_val, val_pred_binary)
    cv_scores.append(accuracy)
    
    # Store OOF predictions
    oof_predictions[val_idx] = val_pred_binary
    
    # Accumulate test predictions
    test_predictions += test_pred / n_splits
    
    print(f"Fold {fold} Accuracy: {accuracy:.4f}")
    
    fold += 1

# Overall CV score
mean_cv_score = np.mean(cv_scores)
std_cv_score = np.std(cv_scores)

print(f"\n{'='*50}")
print(f"Cross-Validation Results:")
print(f"Mean Accuracy: {mean_cv_score:.4f} ± {std_cv_score:.4f}")
print(f"Individual folds: {[f'{score:.4f}' for score in cv_scores]}")
print(f"{'='*50}")

# OOF accuracy
oof_accuracy = accuracy_score(y, oof_predictions)
print(f"OOF Accuracy: {oof_accuracy:.4f}")

Starting 5-fold cross-validation...

Fold 1/5
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[53]	valid's binary_error: 0.117318
Fold 1 Accuracy: 0.8827

Fold 2/5
Training until validation scores don't improve for 50 rounds


Early stopping, best iteration is:
[64]	valid's binary_error: 0.134831
Fold 2 Accuracy: 0.8652

Fold 3/5
Training until validation scores don't improve for 50 rounds


Early stopping, best iteration is:
[15]	valid's binary_error: 0.185393
Fold 3 Accuracy: 0.8146

Fold 4/5


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[11]	valid's binary_error: 0.151685
Fold 4 Accuracy: 0.8483

Fold 5/5
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[59]	valid's binary_error: 0.140449
Fold 5 Accuracy: 0.8596

Cross-Validation Results:
Mean Accuracy: 0.8541 ± 0.0226
Individual folds: ['0.8827', '0.8652', '0.8146', '0.8483', '0.8596']
OOF Accuracy: 0.8541


In [7]:
# Create submission
submission_df = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Survived': (test_predictions > 0.5).astype(int)
})

# Save submission
submission_path = '/home/submission/submission.csv'
submission_df.to_csv(submission_path, index=False)

print(f"Submission saved to: {submission_path}")
print(f"\nSubmission shape: {submission_df.shape}")
print("\nFirst 10 rows of submission:")
print(submission_df.head(10))

# Verify submission format
print(f"\nSubmission columns: {submission_df.columns.tolist()}")
print(f"Unique values in Survived: {submission_df['Survived'].unique()}")
print(f"Value counts:\n{submission_df['Survived'].value_counts()}")

Submission saved to: /home/submission/submission.csv

Submission shape: (418, 2)

First 10 rows of submission:
   PassengerId  Survived
0          892         0
1          893         0
2          894         0
3          895         0
4          896         0
5          897         0
6          898         1
7          899         0
8          900         1
9          901         0

Submission columns: ['PassengerId', 'Survived']
Unique values in Survived: [0 1]
Value counts:
Survived
0    275
1    143
Name: count, dtype: int64


In [8]:
# Feature importance
feature_importance = model.feature_importance(importance_type='gain')
feature_names = X_encoded.columns.tolist()

importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

print("Top 10 Most Important Features:")
print(importance_df.head(10))

# Save results
results = {
    'cv_scores': cv_scores,
    'mean_cv_score': mean_cv_score,
    'std_cv_score': std_cv_score,
    'oof_accuracy': oof_accuracy,
    'feature_importance': importance_df.to_dict('records')
}

import json
with open('/home/code/experiments/001_baseline_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print(f"\nResults saved to: /home/code/experiments/001_baseline_results.json")

Top 10 Most Important Features:
          feature   importance
1             Sex  1524.733730
11  FarePerPerson   602.632620
6             Age   579.309369
9            Fare   539.415274
0          Pclass   453.235429
3           Title   373.418227
5     CabinLetter   221.311582
2        Embarked   136.412549
7           SibSp   100.128826
10     FamilySize    86.340292

Results saved to: /home/code/experiments/001_baseline_results.json
